IMPORTO LIBRERÍAS

In [1]:
from pymongo import MongoClient
import pandas as pd
import pandas as pd
from shapely.geometry import Point

In [35]:
df = pd.read_csv("../Data/df_UberEats.csv", index_col = 0)
df.head(3)

,Restaurante,Valoracion,Nota,Delivery_fee,ETA,Tag,Precio,Direccion,Barrio,Lat,Lon
0,Badsha India Restaurant,200,4.4,1.00,15–35,India,€,"Calle De Embajadores, Madrid",Centro,40.40069,-3.69925
1,TUTTO PASTA!,26,2.8,2.50,40–60,Family Meals,€,"Calle De Núñez De Arce, 11, Madrid",Centro,40.41538,-3.70124
2,Pizza Central,31,3.8,2.99,35–55,Pizza,€,"Plaza Del Emperador Carlos V, 10, Madrid",Centro,40.40861,-3.69288


In [37]:
df['Coordenates'] = df[['Lon', 'Lat']].apply(list, axis=1).tolist()
df.head(3)

,Restaurante,Valoracion,Nota,Delivery_fee,ETA,Tag,Precio,Direccion,Barrio,Lat,Lon,Coordenates
0,Badsha India Restaurant,200,4.4,1.00,15–35,India,€,"Calle De Embajadores, Madrid",Centro,40.40069,-3.69925,"[-3.69925, 40.40069]"
1,TUTTO PASTA!,26,2.8,2.50,40–60,Family Meals,€,"Calle De Núñez De Arce, 11, Madrid",Centro,40.41538,-3.70124,"[-3.70124, 40.41538]"
2,Pizza Central,31,3.8,2.99,35–55,Pizza,€,"Plaza Del Emperador Carlos V, 10, Madrid",Centro,40.40861,-3.69288,"[-3.69288, 40.40861]"


In [38]:
coordenadas = [-73.93, 40.82]

In [39]:
def type_point(lista):
    return {"type":"Point", "coordinates": lista}

In [40]:
punto = type_point(coordenadas)
punto

{'type': 'Point', 'coordinates': [-73.93, 40.82]}

In [41]:
tp = []
for i in df["Coordenates"]:
    tp.append(type_point(i))
df['Coordenates'] = tp

In [42]:
df['Coordenates'][0]

{'type': 'Point', 'coordinates': [-3.69925, 40.40069]}

In [52]:
df.head(5)

,Restaurante,Valoracion,Nota,Delivery_fee,ETA,Tag,Precio,Direccion,Barrio,Lat,Lon,Coordenates
0,Badsha India Restaurant,200,4.4,1.00,15–35,India,€,"Calle De Embajadores, Madrid",Centro,40.40069,-3.69925,"{'type': 'Point', 'coordinates': [-3.69925, 40..."
1,TUTTO PASTA!,26,2.8,2.50,40–60,Family Meals,€,"Calle De Núñez De Arce, 11, Madrid",Centro,40.41538,-3.70124,"{'type': 'Point', 'coordinates': [-3.70124, 40..."
2,Pizza Central,31,3.8,2.99,35–55,Pizza,€,"Plaza Del Emperador Carlos V, 10, Madrid",Centro,40.40861,-3.69288,"{'type': 'Point', 'coordinates': [-3.69288, 40..."
4,La Tagliatella,15,3.9,0.90,25–35,Pizza,€,"Calle De Toledo 27, Madrid",Centro,40.41377,-3.70737,"{'type': 'Point', 'coordinates': [-3.70737, 40..."
5,Fritz Foods,41,4.5,0.40,30–40,Pizza,€,"Calle De Luchana 17, Madrid",Centro,40.43000,-3.70099,"{'type': 'Point', 'coordinates': [-3.70099, 40..."


In [43]:
type_point(df["Coordenates"][0])

{'type': 'Point',
 'coordinates': {'type': 'Point', 'coordinates': [-3.69925, 40.40069]}}

In [45]:
client = MongoClient("localhost:27017")

In [46]:
client = MongoClient("localhost:27017")
db = client.get_database("PROJECT")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PROJECT')

In [47]:
collection = db["DarkKitchens"]

In [48]:
collection.create_index([("type_point_","2dsphere")])

'type_point__2dsphere'

In [49]:
data = df.to_dict(orient = 'records')

In [50]:
data

[{'Restaurante': 'Badsha India Restaurant',
  'Valoracion': 200,
  'Nota': 4.4,
  'Delivery_fee': 1.0,
  'ETA': '15–35',
  'Tag': 'India',
  'Precio': '€',
  'Direccion': 'Calle De Embajadores,   Madrid',
  'Barrio': 'Centro',
  'Lat': 40.40069,
  'Lon': -3.69925,
  'Coordenates': {'type': 'Point', 'coordinates': [-3.69925, 40.40069]}},
 {'Restaurante': 'TUTTO PASTA!',
  'Valoracion': 26,
  'Nota': 2.8,
  'Delivery_fee': 2.5,
  'ETA': '40–60',
  'Tag': 'Family Meals',
  'Precio': '€',
  'Direccion': 'Calle De Núñez De Arce, 11,  Madrid',
  'Barrio': 'Centro',
  'Lat': 40.41538,
  'Lon': -3.70124,
  'Coordenates': {'type': 'Point', 'coordinates': [-3.70124, 40.41538]}},
 {'Restaurante': 'Pizza Central',
  'Valoracion': 31,
  'Nota': 3.8,
  'Delivery_fee': 2.99,
  'ETA': '35–55',
  'Tag': 'Pizza',
  'Precio': '€',
  'Direccion': 'Plaza Del Emperador Carlos V, 10,  Madrid',
  'Barrio': 'Centro',
  'Lat': 40.40861,
  'Lon': -3.69288,
  'Coordenates': {'type': 'Point', 'coordinates': [-3.69

In [51]:
#collection.insert_many(data)